In [1]:
import json
import pandas as pd

pd.set_option("display.max_columns", None)


In [2]:
with open("data/teams.json", "r") as f:
    teams_raw = json.loads(f.read())
    
    
with open("data/clubs.json", "r") as f:
    clubs_raw = json.loads(f.read())


In [3]:
players = []
coaches = []

for team_id in teams_raw:
    for i in range(len(teams_raw[team_id]["players"])):
        
        teams_raw[team_id]["players"][i]["team_id"] = team_id
        teams_raw[team_id]["players"][i]["countryCode"] = teams_raw[team_id]['countryCode']
        players.append(teams_raw[team_id]["players"][i])
        
    for i in range(len(teams_raw[team_id]["coaches"])):        
        
        teams_raw[team_id]["coaches"][i]["team_id"] = team_id
        teams_raw[team_id]["coaches"][i]["countryCode"] = teams_raw[team_id]['countryCode']
        coaches.append( teams_raw[team_id]["coaches"][i] )

In [4]:
import re 

pattern = re.compile(r'(?<!^)(?=[A-Z])')
def snake_case(name):
    name = pattern.sub('_', name).lower()
    return name

In [5]:
players_df = pd.DataFrame(players).astype({"birthDate":"datetime64"})

mapper = {  c:snake_case(c) for c in list(players_df.columns)  }
mapper.update({"fieldPosition_FR":"field_position_fr"})

players_df.rename(columns=mapper, inplace=True)

players_df.head()

,jersey_number,field_position,international_name,birth_date,club_id,detailed_field_position,id,image_url,country_code,field_position_fr,height,weight,team_id
0,1,GOALKEEPER,Salvatore Sirigu,1987-01-12,50029,GOALKEEPER,106737,https://img.uefa.com/imgml/TP/players/3/2020/3...,ITA,Gardien,192.0,80.0,66
1,26,GOALKEEPER,Alex Meret,1997-03-22,50136,GOALKEEPER,250066753,https://img.uefa.com/imgml/TP/players/3/2020/3...,ITA,Gardien,NaN,NaN,66
2,21,GOALKEEPER,Gianluigi Donnarumma,1999-02-25,50058,GOALKEEPER,250078922,https://img.uefa.com/imgml/TP/players/3/2020/3...,ITA,Gardien,196.0,90.0,66
3,25,DEFENDER,Rafael Tolói,1990-10-10,52816,CENTRE_BACK,250112488,https://img.uefa.com/imgml/TP/players/3/2020/3...,ITA,Défenseur,NaN,NaN,66
4,2,DEFENDER,Giovanni Di Lorenzo,1993-08-04,50136,FULL_BACK,250064229,https://img.uefa.com/imgml/TP/players/3/2020/3...,ITA,Défenseur,NaN,NaN,66


In [6]:
clubs = []

for club_id in clubs_raw:
    
    clubs_raw[club_id]["clubId"] = club_id
    clubs.append(clubs_raw[club_id])


clubs_df = pd.DataFrame(clubs)



mapper = {  c:snake_case(c) for c in list(clubs_df.columns)  }
#mapper.update({"fieldPosition_FR":"field_position_fr"})

clubs_df.rename(columns=mapper, inplace=True)
clubs_df.head()

,association_logo_url,big_logo_url,country_code,id_provider,international_name,logo_url,medium_logo_url,team_code,team_type_detail,display_official_name,club_id
0,https://img.uefa.com/imgml/uefacom/elements/lo...,https://img.uefa.com/imgml/TP/teams/logos/700x...,ITA,FAME,Torino,https://img.uefa.com/imgml/TP/teams/logos/70x7...,https://img.uefa.com/imgml/TP/teams/logos/240x...,TOR,DOMESTIC_MEN_TEAM_A,Torino FC,50029
1,https://img.uefa.com/imgml/uefacom/elements/lo...,https://img.uefa.com/imgml/TP/teams/logos/700x...,ITA,FAME,Napoli,https://img.uefa.com/imgml/TP/teams/logos/70x7...,https://img.uefa.com/imgml/TP/teams/logos/240x...,NAP,DOMESTIC_MEN_TEAM_A,SSC Napoli,50136
2,https://img.uefa.com/imgml/uefacom/elements/lo...,https://img.uefa.com/imgml/TP/teams/logos/700x...,ITA,FAME,Milan,https://img.uefa.com/imgml/TP/teams/logos/70x7...,https://img.uefa.com/imgml/TP/teams/logos/240x...,ACM,DOMESTIC_MEN_TEAM_A,AC Milan,50058
3,https://img.uefa.com/imgml/uefacom/elements/lo...,https://img.uefa.com/imgml/TP/teams/logos/700x...,ITA,FAME,Atalanta,https://img.uefa.com/imgml/TP/teams/logos/70x7...,https://img.uefa.com/imgml/TP/teams/logos/240x...,ATA,DOMESTIC_MEN_TEAM_A,Atalanta BC,52816
4,https://img.uefa.com/imgml/uefacom/elements/lo...,https://img.uefa.com/imgml/TP/teams/logos/700x...,ENG,FAME,Chelsea,https://img.uefa.com/imgml/TP/teams/logos/70x7...,https://img.uefa.com/imgml/TP/teams/logos/240x...,CHE,DOMESTIC_MEN_TEAM_A,Chelsea FC,52914


----

##### Some clubs have the same teamCode. Is that a bug ?

In [7]:
#clubs_df.groupby("team_code").count().sort_values(by="club_id", ascending=False).head(15)
#clubs_df[ clubs_df.team_code.isin(["PAR","MON","LEC","NEW","LAR","SHK","CHA","WOL","BAR","LEI"]) ].sort_values(by="team_code")

#####  turns out it's not a bug, it's all normal

----
##### most of the teamTypeDetail is DOMESTIC_MEN_TEAM_A. One is not ...

In [7]:
# most of the teamTypeDetail is DOMESTIC_MEN_TEAM_A. 

# What's the club that is not like it ?
clubs_df[ clubs_df.team_type_detail != "DOMESTIC_MEN_TEAM_A"]

,association_logo_url,big_logo_url,country_code,id_provider,international_name,logo_url,medium_logo_url,team_code,team_type_detail,display_official_name,club_id


In [9]:
# who's the player from that ... Futsal club ?!

players_df[ players_df.club_id == "2604092"]

# Wikipedia tells me he's at Stoke City

,jersey_number,field_position,international_name,birth_date,club_id,detailed_field_position,id,image_url,country_code,field_position_fr,height,weight,team_id


In [10]:
clubs_df[ clubs_df.display_official_name.str.contains("Venezia")].to_records()

rec.array([],
          dtype=[('index', '<i8'), ('association_logo_url', 'O'), ('big_logo_url', 'O'), ('country_code', 'O'), ('id_provider', 'O'), ('international_name', 'O'), ('logo_url', 'O'), ('medium_logo_url', 'O'), ('team_code', 'O'), ('team_type_detail', 'O'), ('display_official_name', 'O'), ('club_id', 'O')])

##### Seems ok but weird though

----

In [8]:
clubs_df.describe()

,association_logo_url,big_logo_url,country_code,id_provider,international_name,logo_url,medium_logo_url,team_code,team_type_detail,display_official_name,club_id
count,210,210,210,210,210,210,210,210,210,210,210
unique,34,210,34,1,210,210,210,178,1,210,210
top,https://img.uefa.com/imgml/uefacom/elements/lo...,https://img.uefa.com/imgml/TP/teams/logos/700x...,ENG,FAME,Helsingborg,https://img.uefa.com/imgml/TP/teams/logos/70x7...,https://img.uefa.com/imgml/TP/teams/logos/240x...,,DOMESTIC_MEN_TEAM_A,Arsenal FC,53357
freq,34,1,34,210,1,1,1,19,210,1,1


---- 
#### Manual edits


##### Players without club

In [9]:
players_df[ players_df["club_id"].isna()  ]

,jersey_number,field_position,international_name,birth_date,club_id,detailed_field_position,id,image_url,country_code,field_position_fr,height,weight,team_id


In [13]:
# Fixes only player without club : David Nemeth from Austria
players_df.loc[ players_df.id == "250150924", ["club_id"] ]  = "50111"


players_df[ players_df.club_id.isna() ]

,jersey_number,field_position,international_name,birth_date,club_id,detailed_field_position,id,image_url,country_code,field_position_fr,height,weight,team_id


##### Player's ages

In [10]:
from datetime import datetime

players_df['age'] = datetime.now().year - players_df.birth_date.dt.year

players_df.head(2)

,jersey_number,field_position,international_name,birth_date,club_id,detailed_field_position,id,image_url,country_code,field_position_fr,height,weight,team_id,age
0,1,GOALKEEPER,Salvatore Sirigu,1987-01-12,50029,GOALKEEPER,106737,https://img.uefa.com/imgml/TP/players/3/2020/3...,ITA,Gardien,192.0,80.0,66,34
1,26,GOALKEEPER,Alex Meret,1997-03-22,50136,GOALKEEPER,250066753,https://img.uefa.com/imgml/TP/players/3/2020/3...,ITA,Gardien,NaN,NaN,66,24


##### Player's BMI

In [11]:
#{\displaystyle IMC=P/T^{2}} (selon l'OMS) ou I M C = m . h − 2 {\displaystyle IMC=m.h^{-2}} {\displaystyle IMC=m.h^{-2}} (selon le système international)
# avec P la masse corporelle (en kilogrammes) et T la stature (en mètres). 

players_df['bmi'] = players_df['weight'] / ( (players_df['height']/100)**2 )

players_df[ ~players_df['bmi'].isna() ].head(2)

,jersey_number,field_position,international_name,birth_date,club_id,detailed_field_position,id,image_url,country_code,field_position_fr,height,weight,team_id,age,bmi
0,1,GOALKEEPER,Salvatore Sirigu,1987-01-12,50029,GOALKEEPER,106737,https://img.uefa.com/imgml/TP/players/3/2020/3...,ITA,Gardien,192.0,80.0,66,34,21.701389
2,21,GOALKEEPER,Gianluigi Donnarumma,1999-02-25,50058,GOALKEEPER,250078922,https://img.uefa.com/imgml/TP/players/3/2020/3...,ITA,Gardien,196.0,90.0,66,22,23.427738


----
### Store the datasets

In [12]:
players_df.to_csv("data/players.csv", index=False)
clubs_df.to_csv("data/clubs.csv", index=False)